In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Carregamento e Pré-Processamento

In [4]:
import pandas as pd

# Definir o caminho do dataset
dataset_path = '/content/drive/MyDrive/LF-Amazon-1.3M/trn.json'

# Ler o arquivo JSON Lines diretamente com pandas
df = pd.read_json(dataset_path, lines=True)

# Remover entradas onde 'title' ou 'content' estão vazios ou nulos
df = df[df['title'].notna() & df['content'].notna()]
df = df[(df['title'].str.strip() != '') & (df['content'].str.strip() != '')]

# Selecionar apenas as colunas 'title' e 'content'
df = df[['title', 'content']]

# Opcional: reduzir o tamanho do dataset para fins de teste
df = df.head(1000)

# Exibir as primeiras linhas do DataFrame
print("Primeiras linhas do DataFrame:")
print(df.head())


Primeiras linhas do DataFrame:
                          title  \
0   Girls Ballet Tutu Neon Pink   
3                 Mog's Kittens   
7   Girls Ballet Tutu Neon Blue   
12                  The Prophet   
13    Rightly Dividing the Word   

                                              content  
0   High quality 3 layer ballet tutu. 12 inches in...  
3   Judith Kerr&#8217;s best&#8211;selling adventu...  
7   Dance tutu for girls ages 2-8 years. Perfect f...  
12  In a distant, timeless place, a mysterious pro...  
13         --This text refers to thePaperbackedition.  


In [5]:
!pip install langchain chromadb sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
!pip install langchain_huggingface

In [9]:
!pip install langchain_chroma

In [12]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.9 MB/s eta 0:00:00


In [14]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.documents import Document

# Usar o modelo de embeddings de sentence-transformers
hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Transformar o DataFrame em uma lista de documentos do LangChain
docs = [Document(page_content=row['content'], metadata={"title": row['title']}) for index, row in df.iterrows()]

# Inicializar o Chroma e criar os embeddings
chroma_vector_store = Chroma.from_documents(docs, hf_embeddings)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
retriever = chroma_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

# Testar uma consulta
query = "Girls Ballet Tutu Neon Pink?"
results = retriever.get_relevant_documents(query)

# Exibir os resultados
for result in results:
    print(f"Title: {result.metadata['title']}\nContent: {result.page_content}\n")


Title: Girls Ballet Tutu Neon Pink
Content: High quality 3 layer ballet tutu. 12 inches in length

